In [1]:
import pandas as pd
import numpy as np

In [2]:
# Importação dos Dados
names=['Data', 'Hora', 'Duração', 'DDD_Origem', 'ID_Origem', 'DDD_Destino', 'ID_Destino', 'Antena', 'Vazio', 'ID_Tipo_Ligação', 'Abrangência_Ligação', 'Operadora_Antena_Origem', 'Operadora_Antena_Destino', 'Categoria_USER', 'X']
dataset = pd.read_csv('../Dados/cdr_Uberlândia.txt', sep=';', names=names)
dataset.head(5)

,Data,Hora,Duração,DDD_Origem,ID_Origem,DDD_Destino,ID_Destino,Antena,Vazio,ID_Tipo_Ligação,Abrangência_Ligação,Operadora_Antena_Origem,Operadora_Antena_Destino,Categoria_USER,X
0,2013-04-16,08:10:26,0.55,84,F28A7CC2643468F06C81510658872E07,84,4EB08BB3316C38C499820B95DF87A725,55061,NaN,117,LOCAL,OI,OI,POS PAGO,300000199
1,2013-04-16,13:38:23,0.77,84,565580EB8FDD16750497A4565ECDBFF8,84,C6C73DC14A20986650521214ADD8537D,55021,NaN,117,LOCAL,OI,OI,PRE PAGO,300000210
2,2013-04-16,09:18:36,4.05,84,565580EB8FDD16750497A4565ECDBFF8,84,C6C73DC14A20986650521214ADD8537D,55021,NaN,117,LOCAL,OI,OI,PRE PAGO,300000210
3,2013-04-16,08:17:51,0.28,34,5E951582ACEB9E81E5786A28866C49CA,34,8CE7D690FAFE920D02B8C67E0E4B3279,55162,NaN,77,LOCAL,OI,CTBC,PRE PAGO,300000210
4,2013-04-16,09:44:52,3.72,84,06D76D98F69BEB0590280A4DC7DF762A,84,3AF1028B2DAFBD6C3590345C089A2B72,55061,NaN,117,LOCAL,OI,OI,PRE PAGO,300000533


In [3]:
# Selecionando somente as instâncias da cidade de Uberlândia
dataset = dataset.query("DDD_Origem == 34 & DDD_Destino == 34")

In [4]:
# Remoção de instâncias duplicadas e colunas inúteis.
dataset.drop_duplicates(inplace=True)
dataset.drop(['Vazio', 'X', 'ID_Tipo_Ligação', 'Abrangência_Ligação', 'DDD_Destino', 'DDD_Origem', 'Operadora_Antena_Origem', 'Operadora_Antena_Destino', 'Categoria_USER'], 
             axis=1, inplace=True)
dataset.head(3)

,Data,Hora,Duração,ID_Origem,ID_Destino,Antena
3,2013-04-16,08:17:51,0.28,5E951582ACEB9E81E5786A28866C49CA,8CE7D690FAFE920D02B8C67E0E4B3279,55162
5,2013-04-16,18:47:31,0.37,BCE396FCD91103EA5F34A75CA2CB30ED,CB9016B4905F230F7342D9F45EC1388B,55003
7,2013-04-16,17:24:23,0.37,78FE6A121B5BE5D5C64FDD03CB51A8BF,13461BE1EA1695078BDB2ADC8FE3D45F,56073


In [5]:
# Selecionando as ligações com duração maiores que 5 segundos
dataset = dataset.query("Duração >= 0.08")
dataset.shape

(8550326, 6)

In [6]:
dataset = dataset.groupby(['ID_Origem', 'ID_Destino']).filter(lambda x: len(x) >= 4)

In [7]:
agrup_origem = dataset.groupby(by='ID_Origem').filter(lambda x: len(x) <= 600)['ID_Origem']
agrup_origem.value_counts()

ID_Origem
73C3FDDC32DCC86A635736D52ED6E926    600
A21FA95345C78B6AB1B13772EFE5F07D    599
337860BFD27EE3BAF887FED29FAAE30C    599
539B963DA8412693185F67D92AC81123    598
4E5062F097341C93B13322EFE5D79FBD    597
                                   ... 
A4D5F7734ED0E0F03F9C9ADB275B7FAB      4
F2EC65F66773BC672355A9D5CE452645      4
EC8F606B76B40524FEB4621B638FCEB7      4
ECD23CCFC5AED7B429C98FF6BF4C691B      4
CEDCFDDBB5B8B16DDADA31B319E292A2      4
Name: count, Length: 96872, dtype: int64

In [8]:
agrup_destino = dataset.groupby(by='ID_Destino').filter(lambda x: len(x) <= 600)['ID_Destino']
agrup_destino.value_counts()

ID_Destino
1A840656D92000C6511A8D4A27195D92    595
7E3CAF35849CC8CFB402CE6C72C7FAE8    591
4B316FC6AD62E411532F19CD97DA490F    585
E34B9D3F30F27CF15B232332A52C5355    585
DA4F8B454B19AE3EB890C37E6A53DE63    583
                                   ... 
925F0DEBE20206DE0C50CC1590EA9888      4
45D4552DC6BED295C0B9210D7422C91D      4
84F37CC8E65AC7E00C072F55944154C8      4
2DF860176FB154C80913633616CDA078      4
F0D8DF483DA7C7D98C864B056424CB88      4
Name: count, Length: 234695, dtype: int64

In [9]:
# Todos os IDs que fizeram ou receberam mais de 4 ligações
nos = np.unique(np.append(agrup_origem, agrup_destino))
len(nos)

254634

In [10]:
dataset = dataset.query("ID_Origem in @nos | ID_Destino in @nos")
dataset.shape

(6038019, 6)

In [11]:
dataset.to_csv('cdr_processed_Uberlândia.csv', index=False)